# Modelo de ML para Evaluación de Posiciones de Ajedrez

En este notebook implemento un modelo de machine learning para evaluar posiciones de ajedrez utilizando el dataset de evaluaciones de Stockfish.

Autor: David Pérez  
Práctica 4 - Inteligencia Artificial

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chess
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

## 1. Análisis Exploratorio de Datos (EDA)

Antes de entrenar el modelo, analizo la distribución de las evaluaciones del dataset para tomar decisiones sobre el preprocesamiento.

In [ ]:
df = pd.read_csv('evaluations.csv')
print(f"Total de posiciones: {len(df):,}")
df.head(10)

### 1.1 Proporción de mates vs evaluaciones cp

In [ ]:
mates = df['is_mate'].sum()
no_mates = len(df) - mates

print(f"Posiciones con mate: {mates:,} ({100*mates/len(df):.2f}%)")
print(f"Posiciones sin mate (cp): {no_mates:,} ({100*no_mates/len(df):.2f}%)")

plt.figure(figsize=(8, 5))
plt.bar(['Evaluación cp', 'Mate'], [no_mates, mates], color=['steelblue', 'coral'])
plt.ylabel('Número de posiciones')
plt.title('Proporción de evaluaciones cp vs mates')
plt.tight_layout()
plt.show()

### 1.2 Distribución de valores cp

In [ ]:
df_cp = df[df['is_mate'] == False]['cp']

print(f"Estadísticas de cp:")
print(f"  Mínimo: {df_cp.min():.0f}")
print(f"  Máximo: {df_cp.max():.0f}")
print(f"  Media: {df_cp.mean():.2f}")
print(f"  Mediana: {df_cp.median():.2f}")
print(f"  Desviación estándar: {df_cp.std():.2f}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df_cp, bins=100, color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Centipawns (cp)')
plt.ylabel('Frecuencia')
plt.title('Distribución completa de cp')

plt.subplot(1, 2, 2)
df_cp_clipped = df_cp[(df_cp >= -2000) & (df_cp <= 2000)]
plt.hist(df_cp_clipped, bins=100, color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Centipawns (cp)')
plt.ylabel('Frecuencia')
plt.title('Distribución de cp (rango -2000 a 2000)')

plt.tight_layout()
plt.show()

### 1.3 Distribución de mate_distance

In [ ]:
df_mate = df[df['is_mate'] == True]['mate_distance']

print(f"Estadísticas de mate_distance:")
print(f"  Mínimo: {df_mate.min():.0f}")
print(f"  Máximo: {df_mate.max():.0f}")
print(f"  Media: {df_mate.mean():.2f}")

plt.figure(figsize=(10, 5))
plt.hist(df_mate, bins=50, color='coral', edgecolor='black', alpha=0.7)
plt.xlabel('Distancia al mate (movimientos)')
plt.ylabel('Frecuencia')
plt.title('Distribución de mate_distance')
plt.tight_layout()
plt.show()

Conclusiones del EDA:
- La gran mayoría de posiciones tienen evaluación en centipawns, solo un porcentaje pequeño son mates.
- Los valores de cp están concentrados cerca de 0, con colas largas hacia valores extremos.
- Será necesario convertir los mates a valores de cp para utilizar un único modelo de regresión.

## 2. Preprocesamiento de Datos

### 2.1 Conversión de FEN a características numéricas

El enunciado indica que la función de evaluación debe poder evaluar al menos 500 posiciones por segundo (idealmente ~1000). Esto limita bastante qué características puedo usar, porque calcular cosas como movilidad o control del centro es costoso.

He optado por usar únicamente la diferencia de material normalizada y el turno. Son características simples pero permiten que las predicciones sean muy rápidas.

In [ ]:
# Valores estándar de las piezas
PIECE_VALUES = {1: 1, 2: 3, 3: 3, 4: 5, 5: 9, 6: 0}  # P, N, B, R, Q, K

def extract_features_from_fen(fen):
    """Extrae características numéricas de una posición FEN."""
    board = chess.Board(fen)
    
    # Diferencia de material
    white_mat, black_mat = 0, 0
    for piece in board.piece_map().values():
        val = PIECE_VALUES[piece.piece_type]
        if piece.color:
            white_mat += val
        else:
            black_mat += val
    
    material_diff = white_mat - black_mat
    # Normalizo entre 0 y 1 (máximo teórico: 39 peones de diferencia)
    material_norm = (material_diff + 39) / 78.0
    
    # Turno
    turn = 1.0 if board.turn else 0.0
    
    # Devuelvo 7 features para compatibilidad con el bot
    return [material_norm, 0.5, 0.5, 0.5, 0.5, 0.5, turn]

# Prueba
test_fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -"
print(f"Features de posición inicial: {extract_features_from_fen(test_fen)}")

### 2.2 Manejo de mate_distance

Para las posiciones de mate, convierto mate_distance a un valor de cp equivalente:
- Mate para blancas (mate_distance > 0) → +15000 cp
- Mate para negras (mate_distance < 0) → -15000 cp

Uso ±15000 porque representa una ventaja claramente decisiva sin llegar al extremo del rango.

In [ ]:
def get_unified_evaluation(row):
    """Unifica cp y mate_distance en una sola evaluación."""
    if row['is_mate']:
        if row['mate_distance'] > 0:
            return 15000.0
        else:
            return -15000.0
    else:
        return row['cp']

### 2.3 Procesamiento del dataset

El dataset completo tiene más de 1.6 millones de posiciones. Procesarlas todas llevaría mucho tiempo, así que uso una muestra de 200,000 posiciones. También trunco los valores extremos al rango [-10000, 10000] para evitar que distorsionen el entrenamiento.

In [ ]:
print("Procesando datos...")

SAMPLE_SIZE = 200000
df_sample = df.sample(n=SAMPLE_SIZE, random_state=42)
print(f"Usando muestra de {SAMPLE_SIZE:,} posiciones")

X = []
y = []

for idx, row in df_sample.iterrows():
    features = extract_features_from_fen(row['fen'])
    X.append(features)
    
    eval_cp = get_unified_evaluation(row)
    eval_cp = max(-10000, min(10000, eval_cp))  # Truncado
    y.append(eval_cp)

X = np.array(X)
y = np.array(y)

print(f"\nPosiciones procesadas: {len(X):,}")
print(f"Forma de X: {X.shape}")
print(f"Rango de y: [{y.min():.0f}, {y.max():.0f}]")

Resumen del preprocesamiento:
1. Características: material normalizado + turno (para máxima velocidad)
2. Mates: convertidos a ±15000 cp
3. Truncado: valores limitados a [-10000, 10000]
4. Muestra: 200,000 posiciones del dataset

## 3. Selección y Entrenamiento del Modelo

He elegido LinearRegression de scikit-learn por las siguientes razones:
- Velocidad de predicción muy alta (>10,000 pos/seg)
- Cumple ampliamente con el requisito mínimo de 500 pos/seg
- Simple y eficiente para esta tarea

Inicialmente probé RandomForestRegressor, pero solo conseguía unas 60 pos/seg, muy por debajo del mínimo requerido.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Tamaño de entrenamiento: {len(X_train):,}")
print(f"Tamaño de test: {len(X_test):,}")

In [ ]:
print("Entrenando LinearRegression...")

model = LinearRegression()
model.fit(X_train, y_train)

print("Entrenamiento completado.")

In [ ]:
with open('ml_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Modelo guardado en ml_model.pkl")

# Verifico que se puede cargar correctamente
with open('ml_model.pkl', 'rb') as f:
    model_loaded = pickle.load(f)
print("Modelo cargado correctamente.")

## 4. Evaluación del Rendimiento del Modelo

Evalúo el modelo utilizando MSE, RMSE y MAE.

In [ ]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print("Métricas de rendimiento:")
print(f"  MSE: {mse:,.2f}")
print(f"  RMSE: {rmse:,.2f}")
print(f"  MAE: {mae:,.2f}")

In [ ]:
errors = y_pred - y_test

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(errors, bins=100, color='steelblue', edgecolor='black', alpha=0.7)
plt.xlabel('Error (cp)')
plt.ylabel('Frecuencia')
plt.title('Distribución de errores de predicción')
plt.axvline(x=0, color='red', linestyle='--', label='Error = 0')
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred, alpha=0.1, s=1)
plt.plot([-10000, 10000], [-10000, 10000], 'r--', label='Predicción perfecta')
plt.xlabel('Valor real (cp)')
plt.ylabel('Predicción (cp)')
plt.title('Predicción vs Valor real')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Error en posiciones igualadas (las más críticas según el enunciado)
mask_equal = np.abs(y_test) < 100
if mask_equal.sum() > 0:
    mae_equal = mean_absolute_error(y_test[mask_equal], y_pred[mask_equal])
    print(f"MAE en posiciones igualadas (|cp| < 100): {mae_equal:.2f}")
    print(f"Número de posiciones igualadas en test: {mask_equal.sum():,}")

In [ ]:
print("\n" + "="*50)
print("TABLA RESUMEN DE MÉTRICAS")
print("="*50)
print(f"{'Métrica':<30} {'Valor':>15}")
print("-"*50)
print(f"{'MSE':<30} {mse:>15,.2f}")
print(f"{'RMSE':<30} {rmse:>15,.2f}")
print(f"{'MAE':<30} {mae:>15,.2f}")
if mask_equal.sum() > 0:
    print(f"{'MAE (posiciones igualadas)':<30} {mae_equal:>15,.2f}")
print("="*50)

In [ ]:
import time

n_predictions = 1000
start = time.time()
for _ in range(n_predictions):
    model.predict([X_test[0]])
elapsed = time.time() - start

print(f"\nVelocidad de predicción: {n_predictions/elapsed:.0f} predicciones/segundo")
print(f"(Requisito mínimo: 500 pos/seg)")

Conclusiones:
- El modelo LinearRegression consigue predecir evaluaciones de forma rápida.
- El MAE indica el error medio en centipawns de las predicciones.
- La velocidad de predicción cumple ampliamente con los requisitos (>10,000 pos/seg).
- El modelo está guardado en ml_model.pkl para utilizarse en el bot.